# Function calling with Azure Cognitive Search

In this notebook, we'll show how to create a simple chatbot to help you find or create a good recipe. We'll create an index in Azure Cognitive Search using [vector search](), and then use use [function calling]() to write queries to the index.

All of the recipes used in this sample were generated by gpt-35-turbo for demo purposes. The recipes are not guaranteed to be safe or taste good so we don't recommend trying them.

In [1]:
import os  
import json  
import openai  
from tenacity import retry, wait_random_exponential, stop_after_attempt  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import Vector  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SearchField,  
    SemanticSettings,  
    VectorSearch,  
    VectorSearchAlgorithmConfiguration,  
)  

In [2]:
# Load config values
with open(r'config.json') as config_file:
    config_details = json.load(config_file)
    
# Configure environment variables for Azure Cognitive Search
service_endpoint = config_details["SEARCH_SERVICE_ENDPOINT"]
index_name = config_details["SEARCH_INDEX_NAME"]
key = config_details["SEARCH_ADMIN_KEY"]
credential = AzureKeyCredential(key)

# Create the Azure Cognitive Search client to issue queries
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)

# Create the index client
index_client = SearchIndexClient(endpoint=service_endpoint, credential=credential)

# Configure OpenAI environment variables
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = config_details['OPENAI_API_BASE']
openai.api_type = "azure"  
openai.api_version = config_details['OPENAI_API_VERSION']

deployment_name = config_details['DEPLOYMENT_NAME'] # You need to use the 0613 version of gpt-35-turbo or gpt-4 to work with functions

## 1.0 Create the search index and load the data

In [3]:
# Create a search index
fields = [
    SimpleField(name="recipe_id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="recipe_category", type=SearchFieldDataType.String, filterable=True, analyzer_name="en.microsoft"),    
    SearchableField(name="recipe_name", type=SearchFieldDataType.String, facetable=True, analyzer_name="en.microsoft"),
    SearchableField(name="ingredients", collection=True, type=SearchFieldDataType.String, facetable=True, filterable=True),
    SearchableField(name="recipe", type=SearchFieldDataType.String, analyzer_name="en.microsoft"),
    SearchableField(name="description", type=SearchFieldDataType.String, analyzer_name="en.microsoft"),
    SimpleField(name="total_time", type=SearchFieldDataType.Int32, filterable=True, facetable=True),
    SearchField(name="recipe_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_configuration="my-vector-config")
]

vector_search = VectorSearch(
    algorithm_configurations=[
        VectorSearchAlgorithmConfiguration(
            name="my-vector-config",
            kind="hnsw",
            hnsw_parameters={
                "m": 4,
                "efConstruction": 400,
                "efSearch": 500,
                "metric": "cosine"
            }
        )
    ]
)

# Semantic Configuration to leverage Bing family of ML models for re-ranking (L2)
semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=PrioritizedFields(
        title_field=None,
        prioritized_keywords_fields=[],
        prioritized_content_fields=[SemanticField(field_name="recipe")]
    ))
semantic_settings = SemanticSettings(configurations=[semantic_config])


# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields, 
                    vector_search=vector_search, semantic_settings=semantic_settings)
result = index_client.delete_index(index)
print(f' {index_name} deleted')
result = index_client.create_index(index)
print(f' {result.name} created')

 recipes-vectors deleted
 recipes-vectors created


### Define a helper function to create embeddings

In [4]:
# Function to generate embeddings for title and content fields, also used for query embeddings
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def generate_embeddings(text):
    response = openai.Embedding.create(
        input=text, engine="text-embedding-ada-002")
    embeddings = response['data'][0]['embedding']
    return embeddings

### Load the data into Azure Cognitive Search

In [5]:
batch_size = 100
counter = 0
documents = []
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)

with open("recipes_final.jsonl", "r") as j_in:
    for line in j_in:
        counter += 1 
        json_recipe = json.loads(line)
        json_recipe['total_time'] = int(json_recipe['total_time'].split(' ')[0])
        json_recipe['recipe_vector'] = generate_embeddings(json_recipe['recipe'])
        json_recipe["@search.action"] = "upload"
        documents.append(json_recipe)
        if counter % batch_size == 0:
            # Load content into index
            result = search_client.upload_documents(documents)  
            print(f"Uploaded {len(documents)} documents") 
            documents = []
            
            
if documents != []:
    # Load content into index
    result = search_client.upload_documents(documents)  
    print(f"Uploaded {len(documents)} documents") 


Uploaded 100 documents
Uploaded 100 documents
Uploaded 100 documents
Uploaded 100 documents
Uploaded 3 documents


## 2.0 Test function calling

In [6]:
messages = [{"role": "user", "content": "Help me find a good lasagna recipe."}]
    
# messages = [{"role": "user", "content": "Help me find a good mexican recipe that has beans and rice"}]
# messages = [{"role": "user", "content": "What should I cook for dinner?"}]

### Try again with a more detailed system message ###
# system_message = """Assistant is a large language model designed to help users find and create recipes.
# You have access to an Azure Cognitive Search index with hundreds of recipes. You can search for recipes by name, ingredient, or cuisine.
# You are designed to be an interactive assistant, so you can ask users clarifying questions to help them find the right recipe. It's better to give more detailed queries to the search index rather than vague one.
# """

# messages = [{"role": "system", "content": system_message},
#             {"role": "user", "content": "What should I cook for dinner?"}]

# messages = [{"role": "system", "content": system_message},
#            {"role": "user", "content": "find an easy mexican recipe with beans and rice"}]
                
functions = [
    {
        "name": "query_recipes",
        "description": "Retrieve recipes from the Azure Cognitive Search index",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The query string to search for recipes",
                },
                "ingredients_filter": {
                    "type": "string",
                    "description": "The odata filter to apply for the ingredients field. Only actual ingredient names should be used in this filter. If you're not sure something is an ingredient, don't include this filter. Example: ingredients/any(i: i eq 'salt' or i eq 'pepper')",
                },
                "time_filter": {
                    "type": "string",
                    "description": "The odata filter to apply for the total_time field. If a user asks for a quick or easy recipe, you should filter down to recipes that will take less than 30 minutes. Example: total_time lt 25",
                }
            },
            "required": ["query"],
        },
    }
]

response = openai.ChatCompletion.create(
    deployment_id="gpt-35-turbo-0613",
    messages=messages,
    functions=functions,
    temperature=0.2,
    function_call="auto", 
)

print(response['choices'][0]['message'])

{
  "role": "assistant",
  "function_call": {
    "name": "query_recipes",
    "arguments": "{\n  \"query\": \"lasagna\"\n}"
  }
}


### Define function to call Azure Cognitive Search

In [7]:
def query_recipes(query, ingredients_filter=None, time_filter=None):
    filter = ""
    if ingredients_filter and time_filter:
        filter = f"{time_filter} and {ingredients_filter}"
    elif ingredients_filter:
        filter = ingredients_filter
    elif time_filter:
        filter = time_filter


    results = search_client.search(  
        query_type="semantic",
        query_language="en-us",
        semantic_configuration_name="my-semantic-config",
        search_text=query,  
        vector=generate_embeddings(query),
        filter=filter,
        top_k=3, 
        vector_fields="recipe_vector",
        select=["recipe_id", "recipe", "recipe_category", "recipe_name", "description"],
        top=3
    )  
   
    n = 1
    recipes_for_prompt = ""
    for result in results:
        recipes_for_prompt += f"Recipe {result['recipe_id']}: {result['recipe_name']}: {result['description']}\n"
        n += 1

    return recipes_for_prompt

## 3.0 Get things running end to end

In [8]:
def run_conversation(messages, functions, available_functions, deployment_id):
    
    # Step 1: send the conversation and available functions to GPT
    response = openai.ChatCompletion.create(
        deployment_id=deployment_id,
        messages=messages,
        functions=functions,
        function_call="auto", 
        temperature=0.2
    )
    response_message = response["choices"][0]["message"]


    # Step 2: check if the model wants to call a function
    if response_message.get("function_call"):
        print("Recommended Function call:")
        print(response_message.get("function_call"))
        print()
        
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        function_name = response_message["function_call"]["name"]
        
        # verify function exists
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        function_to_call = available_functions[function_name]  
        
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = function_to_call(**function_args)
        
        print("Output of function call:")
        print(function_response)
        print()
        
        # Step 4: send the info on the function call and function response to the model
        
        # adding assistant response to messages
        messages.append(
            {
                "role": response_message["role"],
                "function_call": {
                    "name": response_message["function_call"]["name"],
                    "arguments": response_message["function_call"]["arguments"],
                },
                "content": None
            }
        )

        # adding function response to messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        print("Messages in second request:")
        for message in messages:
            print(message)
        print()

        second_response = openai.ChatCompletion.create(
            messages=messages,
            deployment_id=deployment_id
        )  # get a new response from GPT where it can see the function response

        return second_response
    else:
        return response

In [9]:
system_message = """Assistant is a large language model designed to help users find and create recipes.

You have access to an Azure Cognitive Search index with hundreds of recipes. You can search for recipes by name, ingredient, or cuisine.

You are designed to be an interactive assistant, so you can ask users clarifying questions to help them find the right recipe. It's better to give more detailed queries to the search index rather than vague one.
"""

messages = [{"role": "system", "content": system_message},
            {"role": "user", "content": "I want to make a pasta dish that takes less than 60 minutes to make."}]

available_functions = {'query_recipes': query_recipes}

result = run_conversation(messages, functions, available_functions, deployment_name)

print("Final response:")
print(result['choices'][0]['message']['content'])

Recommended Function call:
{
  "name": "query_recipes",
  "arguments": "{\n  \"query\": \"pasta\",\n  \"time_filter\": \"total_time lt 60\"\n}"
}

Output of function call:
Recipe 46: Pesto Pasta: Pesto Pasta is a classic Italian dish that combines al dente pasta with a flavorful sauce made from fresh basil, garlic, pine nuts, Parmesan cheese, and olive oil. It's a versatile and delicious meal that can be enjoyed as a main dish or as a side.
Recipe 76: Tortellini Alfredo: A creamy and delicious pasta dish filled with cheesy tortellini, smothered in a rich alfredo sauce.
Recipe 65: Cacio e Pepe: Cacio e Pepe is a classic Roman dish that translates to "cheese and pepper". It consists of simple ingredients that when combined, create a delicious and comforting pasta dish with a creamy, cheesy, and peppery sauce.


Messages in second request:
{'role': 'system', 'content': "Assistant is a large language model designed to help users find and create recipes.\n\nYou have access to an Azure Cogni

## 4.0 Define additional functions

Now that we have the `query_recipes` function defined, we can add additional functions to add more capabilities.

In [10]:
functions = [
    {
        "name": "query_recipes",
        "description": "Retrieve recipes from the Azure Cognitive Search index",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The query string to search for recipes",
                },
                "time_filter": {
                    "type": "string",
                    "description": "The odata filter to apply for the total_time field. If a user asks for a quick or easy recipe, you should filter down to recipes that will take less than 30 minutes. Example: total_time lt 25",
                }
            },
            "required": ["query"],
        },
    },
    {
        "name": "get_recipe",
        "description": "Gets a recipe based on it's id",
        "parameters": {
            "type": "object",
            "properties": {
                "id": {
                    "type": "string",
                    "description": "The id of a recipe. Usually a number such as 3846",
                }
            },
            "required": ["id"],
        }
    },
    {
        "name": "convert_measurement",
        "description": "converts a measurement from one unit to another for common cooking measurements",
        "parameters": {
            "type": "object",
            "properties": {
                "amount": {
                    "type": "number",
                    "description": "The quantity of the measurement to convert.",
                },
                "from_unit": {
                    "type": "string",
                    "description": "The unit to convert the measurement from. Supported values are tablespoons, teaspoons, cups, and ounces.",
                },
                "to_unit": {
                    "type": "string",
                    "description": "The unit to convert the measurement to. Supported values are tablespoons, teaspoons, cups, and ounces.",
                }
            },
            "required": ["amount", "from_unit", "to_unit"],
        }
    }
]

### Define a function to convert common measurements

In [11]:
def convert_measurement(amount, from_unit, to_unit):
    conversions = {
        "tablespoons": {
            "teaspoons": 3,
            "cups": 1/16,
            "ounces": 1/2.667
        },
        "teaspoons": {
            "tablespoons": 1/3,
            "cups": 1/48,
            "ounces": 1/6
        },
        "cups": {
            "tablespoons": 16,
            "teaspoons": 48,
            "ounces": 8
        },
        "ounces": {
            "tablespoons": 3,
            "teaspoons": 6,
            "cups": 1/8
        }
    }
    if from_unit == to_unit:
        return str(amount) + " " + to_unit
    else:
        conversion_factor = conversions[from_unit][to_unit]
        converted_amount = amount * conversion_factor
        return str(converted_amount) + " " + to_unit

convert_measurement(1, from_unit="tablespoons", to_unit="teaspoons")

'3 teaspoons'

### Define a function to get recipes by id

In [12]:
def get_recipe(id):
    return search_client.get_document(key=id)['recipe']

get_recipe("151")

'Recipe: Malabar Paratha\n\nDescription: Malabar Paratha is a popular Indian flatbread known for its flaky and layered texture. It is perfect for serving alongside curries or as a delicious snack on its own.\n\nCook Time: 10 minutes\nPrep Time: 20 minutes\nTotal Time: 30 minutes\n\nIngredients:\n- 2 cups all-purpose flour\n- 1/2 teaspoon salt\n- 1 tablespoon ghee (clarified butter)\n- 3/4 cup water\n- Additional ghee for brushing\n\nInstructions:\n1. In a large mixing bowl, combine the all-purpose flour and salt. Mix well.\n2. Add the ghee to the flour mixture and mix using your fingertips until the mixture resembles breadcrumbs.\n3. Gradually add water to the mixture while kneading the dough. Continue kneading until a soft and smooth dough is formed. Cover the dough and let it rest for 15 minutes.\n4. After the dough has rested, divide it into small equal-sized balls, approximately golf ball-sized.\n5. Take one dough ball and roll it out into a small circle using a rolling pin.\n6. Br

## 5.0 Test more examples 

In [13]:
available_functions = {'query_recipes': query_recipes, 
                       'get_recipe': get_recipe,    
                       'convert_measurement': convert_measurement}

In [14]:
system_message = """Assistant is a large language model designed to help users find and create recipes.

You have access to an Azure Cognitive Search index with hundreds of recipes. You can search for recipes by name, ingredient, or cuisine.

You are designed to be an interactive assistant, so you can ask users clarifying questions to help them find the right recipe. It's better to give more detailed queries to the search index rather than vague one.
"""

messages = [{"role": "system", "content": system_message},
            {"role": "user", "content": "How many cups is 2 tablespoons of butter?"}]

result = run_conversation(messages, functions, available_functions, deployment_name)

print("Final response:")
print(result['choices'][0]['message']['content'])

Recommended Function call:
{
  "name": "convert_measurement",
  "arguments": "{\n  \"amount\": 2,\n  \"from_unit\": \"tablespoons\",\n  \"to_unit\": \"cups\"\n}"
}

Output of function call:
0.125 cups

Messages in second request:
{'role': 'system', 'content': "Assistant is a large language model designed to help users find and create recipes.\n\nYou have access to an Azure Cognitive Search index with hundreds of recipes. You can search for recipes by name, ingredient, or cuisine.\n\nYou are designed to be an interactive assistant, so you can ask users clarifying questions to help them find the right recipe. It's better to give more detailed queries to the search index rather than vague one.\n"}
{'role': 'user', 'content': 'How many cups is 2 tablespoons of butter?'}
{'role': 'assistant', 'function_call': {'name': 'convert_measurement', 'arguments': '{\n  "amount": 2,\n  "from_unit": "tablespoons",\n  "to_unit": "cups"\n}'}, 'content': None}
{'role': 'function', 'name': 'convert_measure

In [ ]:
system_message = """Assistant is a large language model designed to help users find and create recipes.

You have access to an Azure Cognitive Search index with hundreds of recipes. You can search for recipes by name, ingredient, or cuisine.

You are designed to be an interactive assistant, so you can ask users clarifying questions to help them find the right recipe. It's better to give more detailed queries to the search index rather than vague one.
"""

messages = [{'role': 'system', 'content': system_message},
            {'role': 'user', 'content': 'Help me find a Thai recipe I can cook in less than an hour'},
            {'role': 'assistant', 'function_call': {'name': 'query_recipes', 'arguments': '{\n  "query": "Thai",\n  "time_filter": "total_time lt 60"\n}'}, 'content': None},
            {'role': 'function', 'name': 'query_recipes', 'content': "Recipe 200: Thai Peanut Noodles: Thai Peanut Noodles is a delicious and flavorful dish that combines the creaminess of peanut butter with the tanginess of lime and the heat of chili. This dish is perfect for those who enjoy a balance of sweet, savory, and spicy flavors.\nRecipe 206: Thai Cashew Tofu Stir-Fry: This Thai-inspired stir-fry is packed with flavor, combining crispy tofu, crunchy vegetables, and cashews in a savory sauce. It's a quick and delicious weeknight meal option.\nRecipe 196: Thai Beef Salad: Thai Beef Salad is a refreshing and vibrant dish that combines tender beef with a tangy and spicy dressing, fresh herbs, and colorful vegetables.\n"},
            {'role': 'assistant', 'content': "Here are a few Thai recipes that you can cook in less than an hour:\n\n1. Thai Peanut Noodles: This dish combines the creaminess of peanut butter with the tanginess of lime and the heat of chili. It's a perfect balance of sweet, savory, and spicy flavors.\n\n2. Thai Cashew Tofu Stir-Fry: This stir-fry is packed with flavor, combining crispy tofu, crunchy vegetables, and cashews in a savory sauce. It's a quick and delicious option for a weeknight meal.\n\n3. Thai Beef Salad: This refreshing and vibrant salad combines tender beef with a tangy and spicy dressing, fresh herbs, and colorful vegetables.\n\nLet me know if you'd like more information about any of these recipes!"},
            {'role': 'user', 'content': 'Show me the thai peanut noodles recipe'}
]

result = run_conversation(messages, functions, available_functions, deployment_name)

print("Final response:")
print(result['choices'][0]['message'])